# Yahoo Fantasy Sports API

In [1]:
# Authenticate
from yahoo_oauth import OAuth2
sc = OAuth2(None, None, from_file='../references/oauth2.json')

[2021-01-27 21:47:17,780 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-01-27 21:47:17,795 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 43950.913211107254
[2021-01-27 21:47:17,795 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN HAS EXPIRED
[2021-01-27 21:47:17,796 DEBUG] [yahoo_oauth.oauth.refresh_access_token] REFRESHING TOKEN


In [2]:
# Get Game Id
import yahoo_fantasy_api as yfa
gm = yfa.Game(sc, 'nba')

In [3]:
# Get League Ids I have access to
league_id_list = gm.league_ids(year=2020)
league_id = "".join(str(id) for id in league_id_list)
print(f"The League Id is: {league_id}")

The League Id is: 402.l.55374


In [4]:
# Categories
lg = gm.to_league(league_id)
league_categories = lg.stat_categories()

# Print league categories
print(f'The 9 categories in the league are: '
      f'{[category["display_name"] for category in league_categories]}')

The 9 categories in the league are: ['FG%', 'FT%', '3PTM', 'PTS', 'REB', 'AST', 'ST', 'BLK', 'TO']


In [5]:
# League teams
tms = lg.teams()
tms

{'402.l.55374.t.7': {'team_key': '402.l.55374.t.7',
  'team_id': '7',
  'name': 'Russty Cuntbrooks',
  'url': 'https://basketball.fantasysports.yahoo.com/nba/55374/7',
  'team_logos': [{'team_logo': {'size': 'large',
     'url': 'https://s.yimg.com/cv/apiv2/default/nba/nba_5.png'}}],
  'waiver_priority': 3,
  'number_of_moves': '6',
  'number_of_trades': '1',
  'roster_adds': {'coverage_type': 'week', 'coverage_value': 6, 'value': '0'},
  'league_scoring_type': 'headone',
  'draft_position': 10,
  'has_draft_grade': 0,
  'managers': [{'manager': {'manager_id': '7',
     'nickname': 'Alvaro',
     'guid': '54RYPWSN35JEDM6ERNII5GJLDI',
     'email': 'alvaro1410@yahoo.com',
     'image_url': 'https://s.yimg.com/ag/images/default_user_profile_pic_64sq.jpg'}}]},
 '402.l.55374.t.6': {'team_key': '402.l.55374.t.6',
  'team_id': '6',
  'name': "Nunn of Y'all Betta",
  'is_owned_by_current_login': 1,
  'url': 'https://basketball.fantasysports.yahoo.com/nba/55374/6',
  'team_logos': [{'team_logo

In [7]:
# Single team roster
tm = lg.to_team('402.l.55374.t.6')
tm.roster()


## Team Rosters
This section will build the logic needed to create a dataframe with a teams 15 players and their
respective 9cat scores. The mean 9cat scores will be calculated and appended as a row in the
dataframe.

In [11]:
# Return players for all teams
tms = lg.teams()

for team_id in tms:
    tm = lg.to_team(team_id)
    print(tm.roster())

[{'player_id': 6016, 'name': 'Trae Young', 'status': '', 'position_type': 'P', 'eligible_positions': ['PG', 'G', 'Util'], 'selected_position': 'PG'}, {'player_id': 6019, 'name': 'Collin Sexton', 'status': '', 'position_type': 'P', 'eligible_positions': ['PG', 'SG', 'G', 'Util'], 'selected_position': 'SG'}, {'player_id': 6167, 'name': 'Darius Garland', 'status': '', 'position_type': 'P', 'eligible_positions': ['PG', 'G', 'Util'], 'selected_position': 'G'}, {'player_id': 5475, 'name': 'Kelly Oubre Jr.', 'status': '', 'position_type': 'P', 'eligible_positions': ['SF', 'PF', 'F', 'Util'], 'selected_position': 'SF'}, {'player_id': 6166, 'name': "De'Andre Hunter", 'status': '', 'position_type': 'P', 'eligible_positions': ['SF', 'PF', 'F', 'Util'], 'selected_position': 'PF'}, {'player_id': 5894, 'name': 'Chris Boucher', 'status': '', 'position_type': 'P', 'eligible_positions': ['PF', 'F', 'C', 'Util'], 'selected_position': 'F'}, {'player_id': 5294, 'name': 'Joel Embiid', 'status': '', 'positi

In [14]:
# Player list in single team
tm = lg.to_team('402.l.55374.t.6')
team_roster = tm.roster()
player_in_team_list = [player["name"] for player in team_roster]
player_in_team_list

['Patty Mills',
 'Marcus Smart',
 'Malcolm Brogdon',
 'Jeremy Lamb',
 'Khris Middleton',
 'Duncan Robinson',
 'Pascal Siakam',
 'Nikola Jokic',
 'Eric Bledsoe',
 'Cameron Johnson',
 'Davis Bertans',
 'Enes Kanter',
 'Jimmy Butler',
 'Will Barton',
 'Wayne Ellington']

In [15]:
# Get player 9cat
from src.data import player_9cat_average as p9ca

p9ca.get_player_9cat_season_average("Will Barton")

,PLAYER,FG_PCT,FT_PCT,FG3M,PTS,REB,AST,STL,BLK,TOV,GAMES
Mean,Will Barton,0.405,0.735,1.35,10.82,4.24,3.12,0.94,0.53,1.24,18


In [16]:
import pandas as pd

player_9cat_dataframe = pd.DataFrame()
for player in player_in_team_list:
    player_9cat_dataframe = player_9cat_dataframe.append(p9ca.get_player_9cat_season_average(player))

In [18]:
# Reset index
player_9cat_dataframe.reset_index(inplace=True, drop=True)
player_9cat_dataframe

,PLAYER,FG_PCT,FT_PCT,FG3M,PTS,REB,AST,STL,BLK,TOV,GAMES
0,Patty Mills,0.466,0.909,2.82,14.47,1.76,3.00,0.41,0.06,0.82,18
1,Marcus Smart,0.398,0.742,1.73,13.60,2.73,6.20,1.87,0.53,1.80,16
2,Malcolm Brogdon,0.455,0.900,2.82,22.71,4.12,7.18,1.53,0.18,2.06,18
3,Jeremy Lamb,0.535,0.933,1.75,16.75,4.75,0.75,1.00,0.25,1.00,5
4,Khris Middleton,0.522,0.920,2.50,21.75,6.31,5.75,1.00,0.19,2.31,17
5,Duncan Robinson,0.447,0.879,3.56,14.25,3.94,1.56,0.44,0.38,1.56,17
6,Pascal Siakam,0.442,0.767,1.14,18.57,8.14,4.71,0.86,0.57,2.64,15
7,Nikola Jokic,0.563,0.840,1.06,25.41,11.88,9.29,1.88,0.53,4.18,18
8,Eric Bledsoe,0.441,0.679,1.86,12.79,3.64,3.57,1.00,0.29,1.93,15
9,Cameron Johnson,0.448,0.824,2.40,11.87,3.20,1.53,0.73,0.47,0.80,16


In [19]:
# Change column datatype
for column in player_9cat_dataframe.columns[1:]:
    player_9cat_dataframe[column] = player_9cat_dataframe[column].astype(float)

In [20]:
# Calculate team mean 9cat
player_9cat_dataframe.loc["mean"] = player_9cat_dataframe.mean()
player_9cat_dataframe

,PLAYER,FG_PCT,FT_PCT,FG3M,PTS,REB,AST,STL,BLK,TOV,GAMES
0,Patty Mills,0.466000,0.909000,2.820000,14.470000,1.760,3.00,0.41,0.060000,0.820,18.000000
1,Marcus Smart,0.398000,0.742000,1.730000,13.600000,2.730,6.20,1.87,0.530000,1.800,16.000000
2,Malcolm Brogdon,0.455000,0.900000,2.820000,22.710000,4.120,7.18,1.53,0.180000,2.060,18.000000
3,Jeremy Lamb,0.535000,0.933000,1.750000,16.750000,4.750,0.75,1.00,0.250000,1.000,5.000000
4,Khris Middleton,0.522000,0.920000,2.500000,21.750000,6.310,5.75,1.00,0.190000,2.310,17.000000
5,Duncan Robinson,0.447000,0.879000,3.560000,14.250000,3.940,1.56,0.44,0.380000,1.560,17.000000
6,Pascal Siakam,0.442000,0.767000,1.140000,18.570000,8.140,4.71,0.86,0.570000,2.640,15.000000
7,Nikola Jokic,0.563000,0.840000,1.060000,25.410000,11.880,9.29,1.88,0.530000,4.180,18.000000
8,Eric Bledsoe,0.441000,0.679000,1.860000,12.790000,3.640,3.57,1.00,0.290000,1.930,15.000000
9,Cameron Johnson,0.448000,0.824000,2.400000,11.870000,3.200,1.53,0.73,0.470000,0.800,16.000000


In [1]:
import pandas as pd
from yahoo_oauth import OAuth2
import yahoo_fantasy_api as yfa
from src.data import player_9cat_average as p9ca
from nba_api.stats.endpoints import commonallplayers
from nba_api.stats.endpoints import commonplayerinfo

NBA = "nba"
SEASON = 2020
JSON_FOLDER = "../references/oauth2.json"
# Team dictionaries
LALALAND = {'team_key': '402.l.55374.t.1', 'name': 'LaLaLand'}
AUTOPICK = {'team_key': '402.l.55374.t.2', 'name': 'Autopick'}
CRABBEHERBYTHEPUSSY = {'team_key': '402.l.55374.t.3', 'name': 'CrabbeHerByThePussy'}
MAGICS_JOHNSON = {'team_key': '402.l.55374.t.4', 'name': "Magic's Johnson"}
MCCURRY = {'team_key': '402.l.55374.t.5', 'name': 'McCurry'}
NUNN_OF_YALL_BETTA = {'team_key': '402.l.55374.t.6', 'name': "Nunn of Y'all Betta"}
RUSTY_CUNTBROOKS = {'team_key': '402.l.55374.t.7', 'name': 'Russty Cuntbrooks'}
WAKANDA_FOREVER = {'team_key': '402.l.55374.t.8', 'name': 'Wakanda Forever'}
SWAGGY_P = {'team_key': '402.l.55374.t.9', 'name': 'Swaggy P'}
YOBITCH_TOPPIN_ME = {'team_key': '402.l.55374.t.10', 'name': 'yOBItch Toppin Me'}
TVONS_TIP_TOP_TEAM = {'team_key': '402.l.55374.t.11', 'name': "Tvon's Tip-Top Team"}
EL_LADRON_DE_CABRAS = {'team_key': '402.l.55374.t.12', 'name': 'El Ladrón de Cabras'}
league_team_list = [LALALAND, AUTOPICK, CRABBEHERBYTHEPUSSY, MAGICS_JOHNSON, MCCURRY,
                    NUNN_OF_YALL_BETTA, RUSTY_CUNTBROOKS, WAKANDA_FOREVER, SWAGGY_P,
                    YOBITCH_TOPPIN_ME, TVONS_TIP_TOP_TEAM, EL_LADRON_DE_CABRAS]

NAME_KEY = "name"
GAMES_COLUMN = "GAMES"
MEAN_ROW = "mean"
TEAM_KEY = "team_key"
PLAYER_COLUMN = "PLAYER"
TEAM_COLUMN = "TEAM"
EDITORIAL_TEAM_FULL_NAME = "editorial_team_full_name"
UNIFORM_NUMBER = "uniform_number"
COMMON_ALL_PLAYERS ="CommonAllPlayers"
PERSON_ID_KEY = "PERSON_ID"
TEAM_CITY_KEY = "TEAM_CITY"
TEAM_NAME_KEY = "TEAM_NAME"
COMMON_PLAYER_INFO = "CommonPlayerInfo"
JERSEY_KEY = "JERSEY"
DISPLAY_FIRST_LAST = "DISPLAY_FIRST_LAST"

def yahoo_fantasy_api_authentication():

    sc = OAuth2(None, None, from_file=JSON_FOLDER)

    return sc


def yahoo_fantasy_league(sc):

    gm = yfa.Game(sc, NBA)
    league_id_list = gm.league_ids(year=SEASON)
    league_id = "".join(str(id) for id in league_id_list)
    league = gm.to_league(league_id)

    return league


def yahoo_player_team_and_jersey(player_name):

    player_details_dictionary = league.player_details(player_name)
    players_team = player_details_dictionary[0][EDITORIAL_TEAM_FULL_NAME]
    players_number = player_details_dictionary[0][UNIFORM_NUMBER]

    return players_team, players_number


def player_ids_by_nba_team_name(nba_team):

    common_all_players = commonallplayers.CommonAllPlayers(is_only_current_season=1)
    common_all_players_dict = common_all_players.get_normalized_dict()[COMMON_ALL_PLAYERS]
    player_ids_in_team_list = [player[PERSON_ID_KEY] for player in common_all_players_dict if \
        player[TEAM_CITY_KEY] + " " + player[TEAM_NAME_KEY] == nba_team]

    return player_ids_in_team_list


def nba_player_name_from_jersey_search(team_player_ids, yahoo_jersey_number):

    for id in team_player_ids:
        nba_jersey_number = commonplayerinfo.CommonPlayerInfo(id).get_normalized_dict() \
            [COMMON_PLAYER_INFO][0][JERSEY_KEY]
        if nba_jersey_number == yahoo_jersey_number:
            nba_name = commonplayerinfo.CommonPlayerInfo(id).get_normalized_dict() \
            [COMMON_PLAYER_INFO][0][DISPLAY_FIRST_LAST]

            return nba_name


def yahoo_name_to_nba_name(yahoo_player_name):
    """ Function expectes a Yahoo player name and it will find the players Team Name and Jersey
    Number. It then searches the NBA API for Player IDs belonging to found Team Name. For every
    Player ID it searches their Jersey Number. When there is a match, it returns the NBA Player
    Name.
    """

    player_team, player_number = yahoo_player_team_and_jersey(yahoo_player_name)
    player_ids_in_team_list = player_ids_by_nba_team_name(player_team)
    nba_player_name = nba_player_name_from_jersey_search(player_ids_in_team_list, player_number)

    return nba_player_name


def create_roster_dataframe(player_list):

    active_players = p9ca.nba_active_players()
    roster_dataframe = pd.DataFrame()
    for player in player_list:
        try:
            roster_dataframe = roster_dataframe.append(p9ca.player_average_9cat_stats(player,
                                                                                      active_players))
        except IndexError:
            nba_player_name = yahoo_name_to_nba_name(player)
            roster_dataframe = roster_dataframe.append(p9ca.player_average_9cat_stats(nba_player_name,
                                                                                      active_players))
        except:
            print(f"Can't find player {player}")

    roster_dataframe.reset_index(inplace=True, drop=True)

    return roster_dataframe


def format_roster_dataframe(roster_dataframe):

    for column in roster_dataframe.columns[1:]:
        roster_dataframe[column] = roster_dataframe[column].astype(float)

    roster_dataframe.loc[MEAN_ROW] = roster_dataframe.mean()
    roster_dataframe = p9ca.format_dataframe_decimals(roster_dataframe)
    try:
        roster_dataframe.drop(GAMES_COLUMN, axis=1, inplace=True)
    except:
        pass

    return roster_dataframe


def team_9cat_average_stats(team_dictionary, league):

    team = league.to_team(team_dictionary[TEAM_KEY])
    team_roster = team.roster()
    player_in_team_list = [player[NAME_KEY] for player in team_roster]

    roster_dataframe = create_roster_dataframe(player_in_team_list)

    roster_dataframe = format_roster_dataframe(roster_dataframe)

    return roster_dataframe


def fantasy_team_mean_stats(roster_dataframe, team_dictionary):

    mean_series = roster_dataframe.loc[MEAN_ROW]
    mean_series[PLAYER_COLUMN] = team_dictionary[NAME_KEY]
    team_mean_dataframe = pd.DataFrame(mean_series).T
    team_mean_dataframe = team_mean_dataframe.rename(columns={PLAYER_COLUMN : TEAM_COLUMN})
    team_mean_dataframe.reset_index(inplace=True, drop=True)

    return team_mean_dataframe


def player_to_team_mean_stats(team_dictionary, league):
    """
    This function creates a dataframe containing the season averaged stats for each player on a
    Fantasy Team. It then finds the average stats for the entire team and returns a single row
    dataframe with the Fantasy Team name and its seasoned averaged 9cat stats.
    """

    roster_dataframe = team_9cat_average_stats(team_dictionary, league)
    team_mean_dataframe = fantasy_team_mean_stats(roster_dataframe, team_dictionary)

    return team_mean_dataframe


def league_averages(league_team_list):

    league_averages_dataframe = pd.DataFrame()
    for team in league_team_list:
        yahoo_fantasy_api_authentication()
        league_averages_dataframe = league_averages_dataframe.append(player_to_team_mean_stats
                                                                     (team, league))
    league_averages_dataframe.reset_index(drop=True, inplace=True)
    return league_averages_dataframe

In [12]:
# Single team average
sc = yahoo_fantasy_api_authentication()
league = yahoo_fantasy_league(sc)
team_9cat_average_stats(CRABBEHERBYTHEPUSSY, league)


[2021-01-23 23:07:43,569 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-01-23 23:07:43,573 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 2186.45499253273
[2021-01-23 23:07:43,574 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID


,PLAYER,FG_PCT,FT_PCT,FG3M,PTS,REB,AST,STL,BLK,TOV
0,Chris Paul,0.440,0.700,0.9,13.3,4.4,8.7,0.9,0.3,2.3
1,Norman Powell,0.386,0.470,2.0,12.5,2.1,0.7,1.0,0.2,1.3
2,Deni Avdija,0.459,0.273,1.5,7.0,4.0,2.2,1.0,0.3,0.6
3,Otto Porter Jr.,0.481,0.545,1.8,12.5,6.8,1.9,0.7,0.1,1.1
4,John Collins,0.520,0.554,1.0,15.9,7.5,1.5,0.6,1.0,1.3
5,Tobias Harris,0.526,0.731,2.1,19.6,6.8,3.1,1.1,0.9,2.2
6,Bam Adebayo,0.607,0.858,0.0,18.6,9.2,5.2,0.8,1.0,3.5
7,Karl-Anthony Towns,0.479,0.958,1.5,22.0,12.5,4.2,0.5,2.8,2.5
8,Larry Nance Jr.,0.505,0.378,1.6,11.1,6.5,3.7,2.4,0.5,1.7
9,Nicolas Batum,0.498,0.479,2.1,10.2,4.9,2.5,1.3,0.4,0.7


In [2]:
%%time
# League average
sc = yahoo_fantasy_api_authentication()
league = yahoo_fantasy_league(sc)
league_averages_dataframe = league_averages(league_team_list)
league_averages_dataframe

[2021-01-28 21:51:04,400 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-01-28 21:51:04,402 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 1426.3616027832031
[2021-01-28 21:51:04,403 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2021-01-28 21:51:06,777 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-01-28 21:51:06,787 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 1428.747150182724
[2021-01-28 21:51:06,787 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2021-01-28 21:52:10,307 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-01-28 21:52:10,310 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 1492.269475221634
[2021-01-28 21:52:10,310 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2021-01-28 21:53:14,691 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-01-28 21:53:14,695 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 1556.65443110466
[2021-01-28 21:53:14,695 DEBUG] [yahoo_oauth.oauth.t

Wall time: 12min 16s


,TEAM,FG_PCT,FT_PCT,FG3M,PTS,REB,AST,STL,BLK,TOV
0,LaLaLand,0.461,0.761,1.57,16.22,5.88,3.76,0.83,0.55,2.27
1,Autopick,0.489,0.796,1.62,16.25,5.69,3.29,0.88,0.52,1.86
2,CrabbeHerByThePussy,0.491,0.816,1.47,14.02,6.13,3.08,0.91,0.94,1.59
3,Magic's Johnson,0.448,0.796,2.12,16.45,5.75,3.77,1.02,0.52,1.75
4,McCurry,0.493,0.730,0.87,14.09,6.72,3.56,1.09,0.78,1.88
5,Nunn of Y'all Betta,0.469,0.829,1.96,15.41,5.08,3.71,1.00,0.40,1.62
6,Russty Cuntbrooks,0.474,0.776,1.62,17.19,5.83,3.05,1.01,0.86,1.97
7,Wakanda Forever,0.450,0.723,1.78,15.17,5.41,3.16,0.92,0.57,1.65
8,Swaggy P,0.463,0.742,1.93,14.75,4.95,3.01,0.83,0.60,1.92
9,yOBItch Toppin Me,0.488,0.784,1.73,15.71,4.79,3.34,1.00,0.52,2.06


## Yahoo Player Info
This section will create a function which will retrieve an NBA players team and jersey number.
These identifiers will then be used to find the player in the Stats NBA API. This is done because
 the current method of searching for a players stats on the Stats NBA API using the Yahoo Fantasy
  name can lead to errors. For example, on Yahoo it is PJ Washington, on Stats NBA it is P.J.
  Washington.

In [13]:
sc = yahoo_fantasy_api_authentication()
league = yahoo_fantasy_league(sc)

[2021-01-23 22:33:47,335 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-01-23 22:33:47,337 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 150.21866846084595
[2021-01-23 22:33:47,338 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID


In [54]:
# Function to get team name and jersey
EDITORIAL_TEAM_FULL_NAME = "editorial_team_full_name"
UNIFORM_NUMBER = "uniform_number"

def yahoo_player_team_and_jersey(player_name):

    player_details_dictionary = league.player_details(player_name)
    players_team = player_details_dictionary[0][EDITORIAL_TEAM_FULL_NAME]
    players_number = player_details_dictionary[0][UNIFORM_NUMBER]

    return players_team, players_number

In [46]:
# Get team name and jersey
players_team, players_number = yahoo_player_team_and_jersey("Robert Williams")
print(players_team, players_team)

In [47]:
# Player Ids in team
COMMON_ALL_PLAYERS ="CommonAllPlayers"
PERSON_ID_KEY = "PERSON_ID"
TEAM_CITY_KEY = "TEAM_CITY"
TEAM_NAME_KEY = "TEAM_NAME"

def player_ids_by_nba_team_name(nba_team):

    common_all_players = commonallplayers.CommonAllPlayers(is_only_current_season=1)
    common_all_players_dict = common_all_players.get_normalized_dict()[COMMON_ALL_PLAYERS]
    player_ids_in_team_list = [player[PERSON_ID_KEY] for player in common_all_players_dict if \
        player[TEAM_CITY_KEY] + " " + player[TEAM_NAME_KEY] == players_team]

    return player_ids_in_team_list

In [48]:
# Player Ids
player_ids_in_team_list = player_ids_by_nba_team_name(players_team)
player_ids_in_team_list

In [49]:
# Find name from jersey number
COMMON_PLAYER_INFO = "CommonPlayerInfo"
JERSEY_KEY = "JERSEY"
DISPLAY_FIRST_LAST = "DISPLAY_FIRST_LAST"

def nba_player_name_from_jersey_search(team_player_ids, yahoo_jersey_number):

    for id in team_player_ids:
        nba_jersey_number = commonplayerinfo.CommonPlayerInfo(id).get_normalized_dict() \
            [COMMON_PLAYER_INFO][0][JERSEY_KEY]
        if nba_jersey_number == yahoo_jersey_number:
            nba_name = commonplayerinfo.CommonPlayerInfo(id).get_normalized_dict() \
            [COMMON_PLAYER_INFO][0][DISPLAY_FIRST_LAST]

            return nba_name

In [50]:
# Get NBA Stats name
nba_player_name_from_jersey_search(player_ids_in_team_list, players_number)

'Robert Williams III'

In [23]:
from nba_api.stats.endpoints import commonallplayers
from nba_api.stats.endpoints import commonplayerinfo

EDITORIAL_TEAM_FULL_NAME = "editorial_team_full_name"
UNIFORM_NUMBER = "uniform_number"
COMMON_ALL_PLAYERS ="CommonAllPlayers"
PERSON_ID_KEY = "PERSON_ID"
TEAM_CITY_KEY = "TEAM_CITY"
TEAM_NAME_KEY = "TEAM_NAME"
COMMON_PLAYER_INFO = "CommonPlayerInfo"
JERSEY_KEY = "JERSEY"
DISPLAY_FIRST_LAST = "DISPLAY_FIRST_LAST"
RANK_SUFFIX ="_RANK"


def yahoo_player_team_and_jersey(player_name):

    player_details_dictionary = league.player_details(player_name)
    players_team = player_details_dictionary[0][EDITORIAL_TEAM_FULL_NAME]
    players_number = player_details_dictionary[0][UNIFORM_NUMBER]

    return players_team, players_number


def player_ids_by_nba_team_name(nba_team):

    common_all_players = commonallplayers.CommonAllPlayers(is_only_current_season=1)
    common_all_players_dict = common_all_players.get_normalized_dict()[COMMON_ALL_PLAYERS]
    player_ids_in_team_list = [player[PERSON_ID_KEY] for player in common_all_players_dict if \
        player[TEAM_CITY_KEY] + " " + player[TEAM_NAME_KEY] == nba_team]

    return player_ids_in_team_list


def nba_player_name_from_jersey_search(team_player_ids, yahoo_jersey_number):

    for id in team_player_ids:
        nba_jersey_number = commonplayerinfo.CommonPlayerInfo(id).get_normalized_dict() \
            [COMMON_PLAYER_INFO][0][JERSEY_KEY]
        if nba_jersey_number == yahoo_jersey_number:
            nba_name = commonplayerinfo.CommonPlayerInfo(id).get_normalized_dict() \
            [COMMON_PLAYER_INFO][0][DISPLAY_FIRST_LAST]

            return nba_name


def yahoo_name_to_nba_name(yahoo_player_name):
    """ Function expectes a Yahoo player name and it will find the players Team Name and Jersey
    Number. It then searches the NBA API for Player IDs belonging to found Team Name. For every
    Player ID it searches their Jersey Number. When there is a match, it returns the NBA Player
    Name.
    """

    player_team, player_number = yahoo_player_team_and_jersey(yahoo_player_name)
    player_ids_in_team_list = player_ids_by_nba_team_name(player_team)
    nba_player_name = nba_player_name_from_jersey_search(player_ids_in_team_list, player_number)

    return nba_player_name


In [36]:
# Get NBA Stats name from Yahoo name
sc = yahoo_fantasy_api_authentication()
league = yahoo_fantasy_league(sc)
yahoo_name_to_nba_name("Robert Williams")


[2021-01-27 22:03:30,319 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-01-27 22:03:30,321 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 972.5253331661224
[2021-01-27 22:03:30,322 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID


'Robert Williams III'

## Waiver Wire Adds
This section will simulate the dropping of a current team player and the addition of a free agent
. Initially, a snapshot of the teams current 9cat averages will be calculated and saved. A player
 will be dropped from my team a player will be added and then a snapshot of the new 9cat averages
  will be calculated. This will show how this swap impacts my team at an overall level.

In [41]:
def drop_add_mean_9cat_difference(current_9cat, new_9cat):

    current_9cat = remove_player_and_float_convert(current_9cat)
    new_9cat = remove_player_and_float_convert(new_9cat)

    difference_9cat = new_9cat.subtract(current_9cat)

    return difference_9cat

# Function to return team dataframe with 9cat averages
def remove_mean_and_player(team_players_9cat_stats_dataframe, player_to_drop):

    team_players_9cat_stats_dataframe = team_players_9cat_stats_dataframe.drop(MEAN_ROW)
    team_players_9cat_stats_dataframe = \
        team_players_9cat_stats_dataframe[~team_players_9cat_stats_dataframe[PLAYER_COLUMN]
    .str.contains(player_to_drop)]

    return team_players_9cat_stats_dataframe


def waiver_add_and_drop(fantasy_team, player_to_drop, player_to_add):

    team_players_9cat_stats_dataframe = team_9cat_average_stats(fantasy_team, league)
    current_team_9cat_average_stats = pd.DataFrame(team_players_9cat_stats_dataframe
                                                   .loc[MEAN_ROW]).T.drop(PLAYER_COLUMN, axis=1)

    removed_mean_dropped_player_dataframe = remove_mean_and_player\
        (team_players_9cat_stats_dataframe, player_to_drop)

    added_player_9cat_averages = p9ca.get_player_9cat_season_average(player_to_add)
    new_team_9cat_average_stats_dataframe = removed_mean_dropped_player_dataframe.append\
        (added_player_9cat_averages)
    new_team_9cat_average_stats_dataframe.reset_index(inplace=True, drop=True)

    new_team_9cat_average_stats_dataframe = format_roster_dataframe \
        (new_team_9cat_average_stats_dataframe)

    new_team_9cat_average_stats = pd.DataFrame(new_team_9cat_average_stats_dataframe
                                                   .loc[MEAN_ROW]).T.drop(PLAYER_COLUMN, axis=1)

    return team_players_9cat_stats_dataframe, current_team_9cat_average_stats, \
           new_team_9cat_average_stats_dataframe, new_team_9cat_average_stats

In [38]:
# Authenticate if needed
sc = yahoo_fantasy_api_authentication()
league = yahoo_fantasy_league(sc)

[2021-01-27 22:04:13,787 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-01-27 22:04:13,797 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 1016.0008628368378
[2021-01-27 22:04:13,798 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID


In [39]:
# Add drop player
PLAYER_TO_DROP = "Will Barton"
PLAYER_TO_ADD = "Anthony Davis"
current_players_9cat_averages, current_team_9cat_averages, new_players_9cat_averages, \
    new_team_9cat_averages = waiver_add_and_drop(NUNN_OF_YALL_BETTA, PLAYER_TO_DROP, PLAYER_TO_ADD)

,FG_PCT,FT_PCT,FG3M,PTS,REB,AST,STL,BLK,TOV
mean,0.481,0.826,1.94,16.37,5.35,3.74,1.06,0.50,1.66


In [ ]:
# Current team 9cat
current_team_9cat_averages

In [ ]:
# New team 9cat
new_team_9cat_averages

In [102]:
# Format and find 9cat differences
def remove_player_and_float_convert(dataframe):

    for column in dataframe.columns:
        dataframe[column] = dataframe[column].astype(float)

    return dataframe

def drop_add_mean_9cat_difference(current_9cat, new_9cat):

    current_9cat = remove_player_and_float_convert(current_9cat)
    new_9cat = remove_player_and_float_convert(new_9cat)

    difference_9cat = new_9cat.subtract(current_9cat)

    return difference_9cat

In [103]:
# Difference dataframe
difference_team_9cat_averages = drop_add_mean_9cat_difference(current_team_9cat_averages,
                                                  new_team_9cat_averages)

difference_team_9cat_averages

,FG_PCT,FT_PCT,FG3M,PTS,REB,AST,STL,BLK,TOV
mean,0.008,0.001,-0.03,0.74,0.31,0.03,0.02,0.09,0.04


In [63]:
# Visualise current, new and difference
def visualise_team_9cat_averages(player_to_drop, player_to_add, current_9cat, new_9cat,
                                 difference_9cat, visualise=True):
    if visualise:
        print(f"You are dropping {player_to_drop} and adding {player_to_add}! \n")

        print("The current team 9cat averages are:\n")
        print(current_9cat)
        print("\n")

        print("The new team 9cat averages are:\n")
        print(new_9cat)
        print("\n")

        print("The difference 9cat averages are:\n")
        print(difference_9cat)
        print("\n")
    else:
        pass


In [105]:
# Visualise change
visualise_team_9cat_averages(PLAYER_TO_DROP, PLAYER_TO_ADD, current_team_9cat_averages,
                             new_team_9cat_averages, difference_team_9cat_averages)

You are dropping Will Barton and adding Anthony Davis! 

The current team 9cat averages are:

      FG_PCT  FT_PCT  FG3M    PTS   REB   AST   STL   BLK   TOV
mean   0.473   0.822  1.74  15.46  5.29  3.74  1.04  0.43  1.66


The new team 9cat averages are:

      FG_PCT  FT_PCT  FG3M   PTS  REB   AST   STL   BLK  TOV
mean   0.481   0.823  1.71  16.2  5.6  3.77  1.06  0.52  1.7


The difference 9cat averages are:

      FG_PCT  FT_PCT  FG3M   PTS   REB   AST   STL   BLK   TOV
mean   0.008   0.001 -0.03  0.74  0.31  0.03  0.02  0.09  0.04




In [45]:
# Function for the entire process
def compare_team_9cats_on_transaction(team, player_to_drop, player_to_add, visualise=True):

    current_players_9cat_averages, current_team_9cat_averages, new_players_9cat_averages, \
    new_team_9cat_averages = waiver_add_and_drop(NUNN_OF_YALL_BETTA, PLAYER_TO_DROP, PLAYER_TO_ADD)

    difference_team_9cat_averages = drop_add_mean_9cat_difference(current_team_9cat_averages,
                                                  new_team_9cat_averages)

    visualise_team_9cat_averages(player_to_drop, player_to_add, current_team_9cat_averages,
                             new_team_9cat_averages, difference_team_9cat_averages, visualise)

    return current_players_9cat_averages, current_team_9cat_averages, new_players_9cat_averages, \
           new_team_9cat_averages

In [33]:
# Authenticate if needed
sc = yahoo_fantasy_api_authentication()
league = yahoo_fantasy_league(sc)

[2021-01-26 20:03:38,925 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-01-26 20:03:38,928 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 2276.010263442993
[2021-01-26 20:03:38,929 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID


In [34]:
# Pipelined
current_players_9cat_averages, current_team_9cat_averages, new_players_9cat_averages, \
    new_team_9cat_averages = compare_team_9cats_on_transaction(NUNN_OF_YALL_BETTA, PLAYER_TO_DROP,
                                                           PLAYER_TO_ADD)

NameError: name 'PLAYER_TO_DROP' is not defined

In [37]:
PLAYER_TO_DROP = "Will Barton"
PLAYER_TO_ADD = "Wayne Ellington"
current_players_9cat_averages, current_team_9cat_averages, new_players_9cat_averages, \
    new_team_9cat_averages = compare_team_9cats_on_transaction(NUNN_OF_YALL_BETTA, PLAYER_TO_DROP,
                                                           PLAYER_TO_ADD)

You are dropping Will Barton and adding Wayne Ellington! 

The current team 9cat averages are:

      FG_PCT  FT_PCT  FG3M    PTS   REB   AST   STL   BLK  TOV
mean   0.473   0.828  1.74  15.57  5.42  3.79  1.04  0.43  1.7


The new team 9cat averages are:

      FG_PCT  FT_PCT  FG3M    PTS   REB   AST   STL   BLK   TOV
mean   0.481   0.826  1.89  15.76  5.29  3.66  1.01  0.41  1.67


The difference 9cat averages are:

      FG_PCT  FT_PCT  FG3M   PTS   REB   AST   STL   BLK   TOV
mean   0.008  -0.002  0.15  0.19 -0.13 -0.13 -0.03 -0.02 -0.03




In [11]:
# Function to return team dataframe with 9cat averages
def remove_mean_and_player(team_players_9cat_stats_dataframe, player_to_drop):

    team_players_9cat_stats_dataframe = team_players_9cat_stats_dataframe.drop(MEAN_ROW)
    team_players_9cat_stats_dataframe = \
        team_players_9cat_stats_dataframe[~team_players_9cat_stats_dataframe[PLAYER_COLUMN]
    .str.contains(player_to_drop)]

    return team_players_9cat_stats_dataframe


def waiver_add_and_drop(fantasy_team, player_to_drop, player_to_add):

    team_players_9cat_stats_dataframe = team_9cat_average_stats(fantasy_team, league)
    current_team_9cat_average_stats = pd.DataFrame(team_players_9cat_stats_dataframe
                                                   .loc[MEAN_ROW]).T.drop(PLAYER_COLUMN, axis=1)

    removed_mean_dropped_player_dataframe = remove_mean_and_player\
        (team_players_9cat_stats_dataframe, player_to_drop)

    added_player_9cat_averages = p9ca.get_player_9cat_season_average(player_to_add)
    new_team_9cat_average_stats_dataframe = removed_mean_dropped_player_dataframe.append\
        (added_player_9cat_averages)
    new_team_9cat_average_stats_dataframe.reset_index(inplace=True, drop=True)

    new_team_9cat_average_stats_dataframe = format_roster_dataframe \
        (new_team_9cat_average_stats_dataframe)

    new_team_9cat_average_stats = pd.DataFrame(new_team_9cat_average_stats_dataframe
                                                   .loc[MEAN_ROW]).T.drop(PLAYER_COLUMN, axis=1)

    return team_players_9cat_stats_dataframe, current_team_9cat_average_stats, \
           new_team_9cat_average_stats_dataframe, new_team_9cat_average_stats


# Format and find 9cat differences
def remove_player_and_float_convert(dataframe):

    for column in dataframe.columns:
        dataframe[column] = dataframe[column].astype(float)

    return dataframe


def drop_add_mean_9cat_difference(current_9cat, new_9cat):

    current_9cat = remove_player_and_float_convert(current_9cat)
    new_9cat = remove_player_and_float_convert(new_9cat)

    difference_9cat = new_9cat.subtract(current_9cat)

    return difference_9cat


# Visualise current, new and difference
def visualise_team_9cat_averages(player_to_drop, player_to_add, current_9cat, new_9cat,
                                 difference_9cat, visualise=True):
    if visualise:
        print(f"You are dropping {player_to_drop} and adding {player_to_add}! \n")

        print("The current team 9cat averages are:\n")
        print(current_9cat)
        print("\n")

        print("The new team 9cat averages are:\n")
        print(new_9cat)
        print("\n")

        print("The difference 9cat averages are:\n")
        print(difference_9cat)
        print("\n")
    else:
        pass


# Function for the entire process
def compare_team_9cats_on_transaction(team, player_to_drop, player_to_add, visualise=True):

    current_players_9cat_averages, current_team_9cat_averages, new_players_9cat_averages, \
    new_team_9cat_averages = waiver_add_and_drop(team, player_to_drop, player_to_add)

    difference_team_9cat_averages = drop_add_mean_9cat_difference(current_team_9cat_averages,
                                                  new_team_9cat_averages)

    visualise_team_9cat_averages(player_to_drop, player_to_add, current_team_9cat_averages,
                             new_team_9cat_averages, difference_team_9cat_averages, visualise)

    return current_players_9cat_averages, current_team_9cat_averages, new_players_9cat_averages, \
           new_team_9cat_averages

## Power Rankings
This section will calculate the power rankings for the league or a single team. It will take a
dataframe containing 9cat columns and rows of either players or team. It will sort each column to
 identify which rows have the highest/lowest values and return a dataframe with the columns
 replaced by the original dataframes sorted indexes. This is done to quickly identify what
 columns a team is good or bad at.

In [40]:
# Single team average
sc = yahoo_fantasy_api_authentication()
league = yahoo_fantasy_league(sc)
nunn_yall_betta_team_avg = team_9cat_average_stats(NUNN_OF_YALL_BETTA, league)
nunn_yall_betta_team_avg

[2021-01-27 22:23:14,758 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-01-27 22:23:14,761 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 2156.964426279068
[2021-01-27 22:23:14,762 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID


,PLAYER,FG_PCT,FT_PCT,FG3M,PTS,REB,AST,STL,BLK,TOV
0,Fred VanVleet,0.411,0.848,3.24,19.41,4.47,6.35,1.47,0.53,1.76
1,Tyrese Haliburton,0.504,0.818,2.21,11.43,3.00,4.86,1.21,0.71,1.50
2,LeBron James,0.497,0.713,2.72,25.17,7.94,7.39,0.94,0.44,3.67
3,P.J. Washington,0.426,0.741,1.06,12.24,6.65,2.88,0.94,1.35,2.12
4,Bojan Bogdanovic,0.368,0.822,2.47,13.35,3.76,2.06,0.41,0.06,1.53
5,Mikal Bridges,0.510,0.816,2.60,15.07,5.80,1.80,0.60,1.13,0.53
6,Nikola Vucevic,0.487,0.816,2.78,23.50,11.06,3.67,1.22,0.50,1.50
7,Cody Zeller,0.450,0.727,0.00,6.50,5.25,1.25,0.75,0.50,0.75
8,Luguentz Dort,0.468,0.758,2.19,12.75,3.62,1.19,1.00,0.31,1.19
9,Malik Beasley,0.446,0.907,2.75,19.56,4.88,2.31,0.69,0.25,1.38


In [35]:
# Function power ranking
RANK_SUFFIX ="_RANK"

def power_rankings(dataframe):

    power_ranking_dataframe =  dataframe.copy()
    try:
        power_ranking_dataframe.drop(MEAN_ROW, inplace=True)
    except KeyError:
        pass
    for column in power_ranking_dataframe.columns[1:]:

        sorted_points = power_ranking_dataframe[column].sort_values(ascending=False)
        sorted_points.drop_duplicates(inplace=True)

        sorting_dictionary = dict()

        count = 1
        for point in sorted_points:
            sorting_dictionary[point] = count
            count += 1

        power_ranking_dataframe[column + RANK_SUFFIX] = power_ranking_dataframe[column].map\
            (sorting_dictionary)

    power_ranking_dataframe.drop(dataframe.columns[1:], axis=1, inplace=True)

    return power_ranking_dataframe

In [43]:
power_rankings(nunn_yall_betta_team_avg)


,PLAYER,FG_PCT_RANK,FT_PCT_RANK,FG3M_RANK,PTS_RANK,REB_RANK,AST_RANK,STL_RANK,BLK_RANK,TOV_RANK
0,Fred VanVleet,11,3,1,7,8,2,2,7,6
1,Tyrese Haliburton,3,6,8,15,13,5,6,3,9
2,LeBron James,4,13,4,3,1,1,8,9,1
3,P.J. Washington,9,11,14,12,2,9,8,1,4
4,Bojan Bogdanovic,14,5,6,10,9,12,13,14,8
5,Mikal Bridges,1,7,5,9,3,13,12,2,14
6,Nikola Vucevic,5,7,2,4,15,7,5,8,9
7,Cody Zeller,7,12,15,1,5,14,9,8,13
8,Luguentz Dort,6,10,9,11,12,15,7,10,12
9,Malik Beasley,8,1,3,6,6,10,11,11,10


In [55]:
%%time
# League average
sc = yahoo_fantasy_api_authentication()
league = yahoo_fantasy_league(sc)
league_averages_dataframe = league_averages(league_team_list)
league_averages_dataframe

,TEAM,FG_PCT,FT_PCT,FG3M,PTS,REB,AST,STL,BLK,TOV
0,LaLaLand,0.455,0.765,1.52,15.95,5.90,3.79,0.84,0.56,2.26
1,Autopick,0.494,0.792,1.65,16.47,5.72,3.29,0.88,0.51,1.86
2,CrabbeHerByThePussy,0.492,0.820,1.48,14.03,6.16,3.10,0.93,0.94,1.56
3,Magic's Johnson,0.454,0.793,2.16,16.54,5.67,3.75,1.04,0.53,1.79
4,McCurry,0.489,0.734,0.85,13.96,6.65,3.54,1.07,0.78,1.86
5,Nunn of Y'all Betta,0.473,0.828,1.74,15.57,5.42,3.79,1.04,0.43,1.70
6,Russty Cuntbrooks,0.474,0.770,1.62,17.00,5.85,3.02,1.00,0.85,1.96
7,Wakanda Forever,0.448,0.733,1.79,15.21,5.57,3.25,0.94,0.57,1.66
8,Swaggy P,0.464,0.735,1.89,14.71,4.94,2.97,0.81,0.61,1.95
9,yOBItch Toppin Me,0.492,0.789,1.76,15.83,4.80,3.35,0.94,0.54,2.06


In [48]:
# Add/Drop player
sc = yahoo_fantasy_api_authentication()
league = yahoo_fantasy_league(sc)
current_players_9cat_averages, current_team_9cat_averages, new_players_9cat_averages, \
    new_team_9cat_averages = compare_team_9cats_on_transaction(NUNN_OF_YALL_BETTA, "Will Barton",
                                                           "James Harden", visualise=False)

[2021-01-28 22:30:47,552 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-01-28 22:30:47,553 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 3809.5127120018005
[2021-01-28 22:30:47,554 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN HAS EXPIRED
[2021-01-28 22:30:47,555 DEBUG] [yahoo_oauth.oauth.refresh_access_token] REFRESHING TOKEN


In [83]:
def new_league_9cat_averages(league_average_dataframe, team, new_team_9cat_averages):

    removed_team_dataframe = league_average_dataframe[~league_average_dataframe[TEAM_COLUMN].str\
        .contains(team[NAME_KEY])]

    new_team_9cat_averages[TEAM_COLUMN] = team[NAME_KEY]
    added_team_dataframe = removed_team_dataframe.append(new_team_9cat_averages)
    added_team_dataframe.reset_index(drop=True, inplace=True)

    return added_team_dataframe


In [84]:
# New league average dataframe
new_league_averages_dataframe = new_league_9cat_averages(league_averages_dataframe,
                                                      NUNN_OF_YALL_BETTA, new_team_9cat_averages)
new_league_averages_dataframe

,TEAM,FG_PCT,FT_PCT,FG3M,PTS,REB,AST,STL,BLK,TOV
0,LaLaLand,0.461,0.761,1.57,16.22,5.88,3.76,0.83,0.55,2.27
1,Autopick,0.489,0.796,1.62,16.25,5.69,3.29,0.88,0.52,1.86
2,CrabbeHerByThePussy,0.491,0.816,1.47,14.02,6.13,3.08,0.91,0.94,1.59
3,Magic's Johnson,0.448,0.796,2.12,16.45,5.75,3.77,1.02,0.52,1.75
4,McCurry,0.493,0.730,0.87,14.09,6.72,3.56,1.09,0.78,1.88
5,Russty Cuntbrooks,0.474,0.776,1.62,17.19,5.83,3.05,1.01,0.86,1.97
6,Wakanda Forever,0.450,0.723,1.78,15.17,5.41,3.16,0.92,0.57,1.65
7,Swaggy P,0.463,0.742,1.93,14.75,4.95,3.01,0.83,0.60,1.92
8,yOBItch Toppin Me,0.488,0.784,1.73,15.71,4.79,3.34,1.00,0.52,2.06
9,Tvon's Tip-Top Team,0.468,0.761,1.35,13.73,5.55,3.23,0.81,0.93,1.94


In [85]:
def team_dataframe_column_float(roster_dataframe):

    for column in roster_dataframe.columns[1:]:
        roster_dataframe[column] = roster_dataframe[column].astype(float)

    return roster_dataframe

In [86]:
# Convert columns to float
new_league_averages_dataframe = team_dataframe_column_float(new_league_averages_dataframe)
new_league_averages_dataframe

,TEAM,FG_PCT,FT_PCT,FG3M,PTS,REB,AST,STL,BLK,TOV
0,LaLaLand,0.461,0.761,1.57,16.22,5.88,3.76,0.83,0.55,2.27
1,Autopick,0.489,0.796,1.62,16.25,5.69,3.29,0.88,0.52,1.86
2,CrabbeHerByThePussy,0.491,0.816,1.47,14.02,6.13,3.08,0.91,0.94,1.59
3,Magic's Johnson,0.448,0.796,2.12,16.45,5.75,3.77,1.02,0.52,1.75
4,McCurry,0.493,0.730,0.87,14.09,6.72,3.56,1.09,0.78,1.88
5,Russty Cuntbrooks,0.474,0.776,1.62,17.19,5.83,3.05,1.01,0.86,1.97
6,Wakanda Forever,0.450,0.723,1.78,15.17,5.41,3.16,0.92,0.57,1.65
7,Swaggy P,0.463,0.742,1.93,14.75,4.95,3.01,0.83,0.60,1.92
8,yOBItch Toppin Me,0.488,0.784,1.73,15.71,4.79,3.34,1.00,0.52,2.06
9,Tvon's Tip-Top Team,0.468,0.761,1.35,13.73,5.55,3.23,0.81,0.93,1.94


In [87]:
# Power rankings
new_league_power_rankings = power_rankings(new_league_averages_dataframe).sort_values(TEAM_COLUMN)
new_league_power_rankings

,TEAM,FG_PCT_RANK,FT_PCT_RANK,FG3M_RANK,PTS_RANK,REB_RANK,AST_RANK,STL_RANK,BLK_RANK,TOV_RANK
1,Autopick,4,3,6,4,6,6,7,9,7
2,CrabbeHerByThePussy,3,2,9,11,2,10,6,1,12
10,El Ladrón de Cabras,1,7,7,9,7,9,8,5,9
0,LaLaLand,10,6,8,5,3,3,9,8,1
3,Magic's Johnson,12,3,1,2,5,2,2,9,10
4,McCurry,2,9,11,10,1,4,1,4,6
11,Nunn of Y'all Betta,7,1,2,3,10,1,3,10,8
5,Russty Cuntbrooks,6,5,6,1,4,11,3,3,3
7,Swaggy P,9,8,3,8,11,12,9,6,5
9,Tvon's Tip-Top Team,8,6,10,12,8,7,10,2,4


In [88]:
# Get current rankings
current_league_power_rankings = power_rankings(league_averages_dataframe).sort_values(TEAM_COLUMN)
current_league_power_rankings


,TEAM,FG_PCT_RANK,FT_PCT_RANK,FG3M_RANK,PTS_RANK,REB_RANK,AST_RANK,STL_RANK,BLK_RANK,TOV_RANK
1,Autopick,4,3,6,3,6,6,7,9,7
2,CrabbeHerByThePussy,3,2,9,11,2,10,6,1,12
11,El Ladrón de Cabras,1,7,7,9,7,9,8,5,8
0,LaLaLand,10,6,8,4,3,2,9,8,1
3,Magic's Johnson,12,3,1,2,5,1,2,9,9
4,McCurry,2,9,11,10,1,4,1,4,6
5,Nunn of Y'all Betta,7,1,2,6,10,3,4,10,11
6,Russty Cuntbrooks,6,5,6,1,4,11,3,3,3
8,Swaggy P,9,8,3,8,11,12,9,6,5
10,Tvon's Tip-Top Team,8,6,10,12,8,7,10,2,4


In [91]:
# Drop Team column
new_league_power_rankings = new_league_power_rankings.reset_index(drop=True)
new_league_power_rankings = new_league_power_rankings.drop(TEAM_COLUMN, axis=1)
# current_league_power_rankings.reset_index(drop=True).drop(TEAM_COLUMN, inplace=True, axis=1)
new_league_power_rankings

,FG_PCT_RANK,FT_PCT_RANK,FG3M_RANK,PTS_RANK,REB_RANK,AST_RANK,STL_RANK,BLK_RANK,TOV_RANK
0,4,3,6,4,6,6,7,9,7
1,3,2,9,11,2,10,6,1,12
2,1,7,7,9,7,9,8,5,9
3,10,6,8,5,3,3,9,8,1
4,12,3,1,2,5,2,2,9,10
5,2,9,11,10,1,4,1,4,6
6,7,1,2,3,10,1,3,10,8
7,6,5,6,1,4,11,3,3,3
8,9,8,3,8,11,12,9,6,5
9,8,6,10,12,8,7,10,2,4


In [92]:
current_league_power_rankings = current_league_power_rankings.reset_index(drop=True)
current_league_power_rankings = current_league_power_rankings.drop(TEAM_COLUMN, axis=1)
# current_league_power_rankings.reset_index(drop=True).drop(TEAM_COLUMN, inplace=True, axis=1)
current_league_power_rankings

,FG_PCT_RANK,FT_PCT_RANK,FG3M_RANK,PTS_RANK,REB_RANK,AST_RANK,STL_RANK,BLK_RANK,TOV_RANK
0,4,3,6,3,6,6,7,9,7
1,3,2,9,11,2,10,6,1,12
2,1,7,7,9,7,9,8,5,8
3,10,6,8,4,3,2,9,8,1
4,12,3,1,2,5,1,2,9,9
5,2,9,11,10,1,4,1,4,6
6,7,1,2,6,10,3,4,10,11
7,6,5,6,1,4,11,3,3,3
8,9,8,3,8,11,12,9,6,5
9,8,6,10,12,8,7,10,2,4


In [93]:
# Difference
power_rankings_difference = new_league_power_rankings.subtract(current_league_power_rankings)
power_rankings_difference

,FG_PCT_RANK,FT_PCT_RANK,FG3M_RANK,PTS_RANK,REB_RANK,AST_RANK,STL_RANK,BLK_RANK,TOV_RANK
0,0,0,0,1,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,1
3,0,0,0,1,0,1,0,0,0
4,0,0,0,0,0,1,0,0,1
5,0,0,0,0,0,0,0,0,0
6,0,0,0,-3,0,-2,-1,0,-3
7,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0


In [94]:
teams = list(league_averages_dataframe.sort_values(TEAM_COLUMN)[TEAM_COLUMN])
teams


['Autopick',
 'CrabbeHerByThePussy',
 'El Ladrón de Cabras',
 'LaLaLand',
 "Magic's Johnson",
 'McCurry',
 "Nunn of Y'all Betta",
 'Russty Cuntbrooks',
 'Swaggy P',
 "Tvon's Tip-Top Team",
 'Wakanda Forever',
 'yOBItch Toppin Me']

In [95]:
power_rankings_difference[TEAM_COLUMN] = teams
power_rankings_difference

,FG_PCT_RANK,FT_PCT_RANK,FG3M_RANK,PTS_RANK,REB_RANK,AST_RANK,STL_RANK,BLK_RANK,TOV_RANK,TEAM
0,0,0,0,1,0,0,0,0,0,Autopick
1,0,0,0,0,0,0,0,0,0,CrabbeHerByThePussy
2,0,0,0,0,0,0,0,0,1,El Ladrón de Cabras
3,0,0,0,1,0,1,0,0,0,LaLaLand
4,0,0,0,0,0,1,0,0,1,Magic's Johnson
5,0,0,0,0,0,0,0,0,0,McCurry
6,0,0,0,-3,0,-2,-1,0,-3,Nunn of Y'all Betta
7,0,0,0,0,0,0,0,0,0,Russty Cuntbrooks
8,0,0,0,0,0,0,0,0,0,Swaggy P
9,0,0,0,0,0,0,0,0,0,Tvon's Tip-Top Team


In [96]:
columns = list(power_rankings_difference.columns)
columns = [columns[-1]] + columns[:-1]
power_rankings_difference = power_rankings_difference[columns]
power_rankings_difference

In [98]:
power_rankings_difference

def new_league_9cat_averages(league_average_dataframe, team, new_team_9cat_averages):

    removed_team_dataframe = league_average_dataframe[~league_average_dataframe[TEAM_COLUMN].str\
        .contains(team[NAME_KEY])]

    new_team_9cat_averages[TEAM_COLUMN] = team[NAME_KEY]
    added_team_dataframe = removed_team_dataframe.append(new_team_9cat_averages)
    added_team_dataframe.reset_index(drop=True, inplace=True)

    return added_team_dataframe


def team_dataframe_column_float(roster_dataframe):

    for column in roster_dataframe.columns[1:]:
        roster_dataframe[column] = roster_dataframe[column].astype(float)

    return roster_dataframe


def change_in_power_rankings(league_averages_dataframe, team, player_to_drop, player_to_add):

    current_players_9cat_averages, current_team_9cat_averages, new_players_9cat_averages, \
    new_team_9cat_averages = compare_team_9cats_on_transaction(team, player_to_drop,
                                                           player_to_add, visualise=False)

    # New league average dataframe
    new_league_averages_dataframe = new_league_9cat_averages(league_averages_dataframe,
                                                          team, new_team_9cat_averages)

    new_league_averages_dataframe = team_dataframe_column_float(new_league_averages_dataframe)

    # Get new rankings
    new_league_power_rankings = power_rankings(new_league_averages_dataframe).sort_values(TEAM_COLUMN)
    # Get current rankings
    current_league_power_rankings = power_rankings(league_averages_dataframe).sort_values(TEAM_COLUMN)

    # Drop Team column
    new_league_power_rankings = new_league_power_rankings.reset_index(drop=True)
    new_league_power_rankings = new_league_power_rankings.drop(TEAM_COLUMN, axis=1)
    current_league_power_rankings = current_league_power_rankings.reset_index(drop=True)
    current_league_power_rankings = current_league_power_rankings.drop(TEAM_COLUMN, axis=1)

    # Difference
    power_rankings_difference = new_league_power_rankings.subtract(current_league_power_rankings)

    # Sorted teams list
    teams = list(league_averages_dataframe.sort_values(TEAM_COLUMN)[TEAM_COLUMN])

    # Add teams column
    power_rankings_difference[TEAM_COLUMN] = teams

    # Rearrange columns
    columns = list(power_rankings_difference.columns)
    columns = [columns[-1]] + columns[:-1]
    power_rankings_difference = power_rankings_difference[columns]

    return power_rankings_difference

In [99]:
change_in_power_rankings(league_averages_dataframe, NUNN_OF_YALL_BETTA, "Will Barton",
                         "James Harden")

,TEAM,FG_PCT_RANK,FT_PCT_RANK,FG3M_RANK,PTS_RANK,REB_RANK,AST_RANK,STL_RANK,BLK_RANK,TOV_RANK
0,Autopick,0,0,0,1,0,0,0,0,0
1,CrabbeHerByThePussy,0,0,0,0,0,0,0,0,0
2,El Ladrón de Cabras,0,0,0,0,0,0,0,0,1
3,LaLaLand,0,0,0,1,0,1,0,0,0
4,Magic's Johnson,0,0,0,0,0,1,0,0,1
5,McCurry,0,0,0,0,0,0,0,0,0
6,Nunn of Y'all Betta,0,0,0,-3,0,-2,-1,0,-3
7,Russty Cuntbrooks,0,0,0,0,0,0,0,0,0
8,Swaggy P,0,0,0,0,0,0,0,0,0
9,Tvon's Tip-Top Team,0,0,0,0,0,0,0,0,0


## Notebook Functions

In [6]:
import pandas as pd
from yahoo_oauth import OAuth2
import yahoo_fantasy_api as yfa
from src.data import player_9cat_average as p9ca
from nba_api.stats.endpoints import commonallplayers
from nba_api.stats.endpoints import commonplayerinfo


NBA = "nba"
SEASON = 2020
JSON_FOLDER = "../references/oauth2.json"
NAME_KEY = "name"
GAMES_COLUMN = "GAMES"
MEAN_ROW = "mean"
TEAM_KEY = "team_key"
PLAYER_COLUMN = "PLAYER"
TEAM_COLUMN = "TEAM"
EDITORIAL_TEAM_FULL_NAME = "editorial_team_full_name"
UNIFORM_NUMBER = "uniform_number"
COMMON_ALL_PLAYERS ="CommonAllPlayers"
PERSON_ID_KEY = "PERSON_ID"
TEAM_CITY_KEY = "TEAM_CITY"
TEAM_NAME_KEY = "TEAM_NAME"
COMMON_PLAYER_INFO = "CommonPlayerInfo"
JERSEY_KEY = "JERSEY"
DISPLAY_FIRST_LAST = "DISPLAY_FIRST_LAST"
EDITORIAL_TEAM_FULL_NAME = "editorial_team_full_name"
UNIFORM_NUMBER = "uniform_number"
COMMON_ALL_PLAYERS ="CommonAllPlayers"
PERSON_ID_KEY = "PERSON_ID"
TEAM_CITY_KEY = "TEAM_CITY"
TEAM_NAME_KEY = "TEAM_NAME"
COMMON_PLAYER_INFO = "CommonPlayerInfo"
JERSEY_KEY = "JERSEY"
DISPLAY_FIRST_LAST = "DISPLAY_FIRST_LAST"
RANK_SUFFIX ="_RANK"


# Team dictionaries
LALALAND = {'team_key': '402.l.55374.t.1', 'name': 'LaLaLand'}
AUTOPICK = {'team_key': '402.l.55374.t.2', 'name': 'Autopick'}
CRABBEHERBYTHEPUSSY = {'team_key': '402.l.55374.t.3', 'name': 'CrabbeHerByThePussy'}
MAGICS_JOHNSON = {'team_key': '402.l.55374.t.4', 'name': "Magic's Johnson"}
MCCURRY = {'team_key': '402.l.55374.t.5', 'name': 'McCurry'}
NUNN_OF_YALL_BETTA = {'team_key': '402.l.55374.t.6', 'name': "Nunn of Y'all Betta"}
RUSTY_CUNTBROOKS = {'team_key': '402.l.55374.t.7', 'name': 'Russty Cuntbrooks'}
WAKANDA_FOREVER = {'team_key': '402.l.55374.t.8', 'name': 'Wakanda Forever'}
SWAGGY_P = {'team_key': '402.l.55374.t.9', 'name': 'Swaggy P'}
YOBITCH_TOPPIN_ME = {'team_key': '402.l.55374.t.10', 'name': 'yOBItch Toppin Me'}
TVONS_TIP_TOP_TEAM = {'team_key': '402.l.55374.t.11', 'name': "Tvon's Tip-Top Team"}
EL_LADRON_DE_CABRAS = {'team_key': '402.l.55374.t.12', 'name': 'El Ladrón de Cabras'}
league_team_list = [LALALAND, AUTOPICK, CRABBEHERBYTHEPUSSY, MAGICS_JOHNSON, MCCURRY,
                    NUNN_OF_YALL_BETTA, RUSTY_CUNTBROOKS, WAKANDA_FOREVER, SWAGGY_P,
                    YOBITCH_TOPPIN_ME, TVONS_TIP_TOP_TEAM, EL_LADRON_DE_CABRAS]


def yahoo_fantasy_api_authentication():

    sc = OAuth2(None, None, from_file=JSON_FOLDER)

    return sc


def yahoo_fantasy_league(sc):

    gm = yfa.Game(sc, NBA)
    league_id_list = gm.league_ids(year=SEASON)
    league_id = "".join(str(id) for id in league_id_list)
    league = gm.to_league(league_id)

    return league


def yahoo_player_team_and_jersey(player_name):

    player_details_dictionary = league.player_details(player_name)
    players_team = player_details_dictionary[0][EDITORIAL_TEAM_FULL_NAME]
    players_number = player_details_dictionary[0][UNIFORM_NUMBER]

    return players_team, players_number


def player_ids_by_nba_team_name(nba_team):

    common_all_players = commonallplayers.CommonAllPlayers(is_only_current_season=1)
    common_all_players_dict = common_all_players.get_normalized_dict()[COMMON_ALL_PLAYERS]
    player_ids_in_team_list = [player[PERSON_ID_KEY] for player in common_all_players_dict if \
        player[TEAM_CITY_KEY] + " " + player[TEAM_NAME_KEY] == nba_team]

    return player_ids_in_team_list


def nba_player_name_from_jersey_search(team_player_ids, yahoo_jersey_number):

    for id in team_player_ids:
        nba_jersey_number = commonplayerinfo.CommonPlayerInfo(id).get_normalized_dict() \
            [COMMON_PLAYER_INFO][0][JERSEY_KEY]
        if nba_jersey_number == yahoo_jersey_number:
            nba_name = commonplayerinfo.CommonPlayerInfo(id).get_normalized_dict() \
            [COMMON_PLAYER_INFO][0][DISPLAY_FIRST_LAST]

            return nba_name


def yahoo_name_to_nba_name(yahoo_player_name):
    """ Function expectes a Yahoo player name and it will find the players Team Name and Jersey
    Number. It then searches the NBA API for Player IDs belonging to found Team Name. For every
    Player ID it searches their Jersey Number. When there is a match, it returns the NBA Player
    Name.
    """

    player_team, player_number = yahoo_player_team_and_jersey(yahoo_player_name)
    player_ids_in_team_list = player_ids_by_nba_team_name(player_team)
    nba_player_name = nba_player_name_from_jersey_search(player_ids_in_team_list, player_number)

    return nba_player_name


def create_roster_dataframe(player_list):

    active_players = p9ca.nba_active_players()
    roster_dataframe = pd.DataFrame()
    for player in player_list:
        try:
            roster_dataframe = roster_dataframe.append(p9ca.player_average_9cat_stats(player,
                                                                                      active_players))
        except IndexError:
            nba_player_name = yahoo_name_to_nba_name(player)
            roster_dataframe = roster_dataframe.append(p9ca.player_average_9cat_stats(nba_player_name,
                                                                                      active_players))
        except:
            print(f"Can't find player {player}")

    roster_dataframe.reset_index(inplace=True, drop=True)

    return roster_dataframe


def format_roster_dataframe(roster_dataframe):

    for column in roster_dataframe.columns[1:]:
        roster_dataframe[column] = roster_dataframe[column].astype(float)

    roster_dataframe.loc[MEAN_ROW] = roster_dataframe.mean()
    roster_dataframe = p9ca.format_dataframe_decimals(roster_dataframe)
    try:
        roster_dataframe.drop(GAMES_COLUMN, axis=1, inplace=True)
    except:
        pass

    return roster_dataframe


def team_9cat_average_stats(team_dictionary, league):

    team = league.to_team(team_dictionary[TEAM_KEY])
    team_roster = team.roster()
    player_in_team_list = [player[NAME_KEY] for player in team_roster]

    roster_dataframe = create_roster_dataframe(player_in_team_list)

    roster_dataframe = format_roster_dataframe(roster_dataframe)

    return roster_dataframe


def fantasy_team_mean_stats(roster_dataframe, team_dictionary):

    mean_series = roster_dataframe.loc[MEAN_ROW]
    mean_series[PLAYER_COLUMN] = team_dictionary[NAME_KEY]
    team_mean_dataframe = pd.DataFrame(mean_series).T
    team_mean_dataframe = team_mean_dataframe.rename(columns={PLAYER_COLUMN : TEAM_COLUMN})
    team_mean_dataframe.reset_index(inplace=True, drop=True)

    return team_mean_dataframe


def player_to_team_mean_stats(team_dictionary, league):
    """
    This function creates a dataframe containing the season averaged stats for each player on a
    Fantasy Team. It then finds the average stats for the entire team and returns a single row
    dataframe with the Fantasy Team name and its seasoned averaged 9cat stats.
    """

    roster_dataframe = team_9cat_average_stats(team_dictionary, league)
    team_mean_dataframe = fantasy_team_mean_stats(roster_dataframe, team_dictionary)

    return team_mean_dataframe


def league_averages(league_team_list):

    league_averages_dataframe = pd.DataFrame()
    for team in league_team_list:
        yahoo_fantasy_api_authentication()
        league_averages_dataframe = league_averages_dataframe.append(player_to_team_mean_stats
                                                                     (team, league))
    league_averages_dataframe.reset_index(drop=True, inplace=True)
    return league_averages_dataframe


def yahoo_player_team_and_jersey(player_name):

    player_details_dictionary = league.player_details(player_name)
    players_team = player_details_dictionary[0][EDITORIAL_TEAM_FULL_NAME]
    players_number = player_details_dictionary[0][UNIFORM_NUMBER]

    return players_team, players_number


def player_ids_by_nba_team_name(nba_team):

    common_all_players = commonallplayers.CommonAllPlayers(is_only_current_season=1)
    common_all_players_dict = common_all_players.get_normalized_dict()[COMMON_ALL_PLAYERS]
    player_ids_in_team_list = [player[PERSON_ID_KEY] for player in common_all_players_dict if \
        player[TEAM_CITY_KEY] + " " + player[TEAM_NAME_KEY] == nba_team]

    return player_ids_in_team_list


def nba_player_name_from_jersey_search(team_player_ids, yahoo_jersey_number):

    for id in team_player_ids:
        nba_jersey_number = commonplayerinfo.CommonPlayerInfo(id).get_normalized_dict() \
            [COMMON_PLAYER_INFO][0][JERSEY_KEY]
        if nba_jersey_number == yahoo_jersey_number:
            nba_name = commonplayerinfo.CommonPlayerInfo(id).get_normalized_dict() \
            [COMMON_PLAYER_INFO][0][DISPLAY_FIRST_LAST]

            return nba_name


def yahoo_name_to_nba_name(yahoo_player_name):
    """ Function expectes a Yahoo player name and it will find the players Team Name and Jersey
    Number. It then searches the NBA API for Player IDs belonging to found Team Name. For every
    Player ID it searches their Jersey Number. When there is a match, it returns the NBA Player
    Name.
    """

    player_team, player_number = yahoo_player_team_and_jersey(yahoo_player_name)
    player_ids_in_team_list = player_ids_by_nba_team_name(player_team)
    nba_player_name = nba_player_name_from_jersey_search(player_ids_in_team_list, player_number)

    return nba_player_name


# Function to return team dataframe with 9cat averages
def remove_mean_and_player(team_players_9cat_stats_dataframe, player_to_drop):

    team_players_9cat_stats_dataframe = team_players_9cat_stats_dataframe.drop(MEAN_ROW)
    team_players_9cat_stats_dataframe = \
        team_players_9cat_stats_dataframe[~team_players_9cat_stats_dataframe[PLAYER_COLUMN]
    .str.contains(player_to_drop)]

    return team_players_9cat_stats_dataframe


def waiver_add_and_drop(fantasy_team, player_to_drop, player_to_add):

    team_players_9cat_stats_dataframe = team_9cat_average_stats(fantasy_team, league)
    current_team_9cat_average_stats = pd.DataFrame(team_players_9cat_stats_dataframe
                                                   .loc[MEAN_ROW]).T.drop(PLAYER_COLUMN, axis=1)

    removed_mean_dropped_player_dataframe = remove_mean_and_player\
        (team_players_9cat_stats_dataframe, player_to_drop)

    added_player_9cat_averages = p9ca.get_player_9cat_season_average(player_to_add)
    new_team_9cat_average_stats_dataframe = removed_mean_dropped_player_dataframe.append\
        (added_player_9cat_averages)
    new_team_9cat_average_stats_dataframe.reset_index(inplace=True, drop=True)

    new_team_9cat_average_stats_dataframe = format_roster_dataframe \
        (new_team_9cat_average_stats_dataframe)

    new_team_9cat_average_stats = pd.DataFrame(new_team_9cat_average_stats_dataframe
                                                   .loc[MEAN_ROW]).T.drop(PLAYER_COLUMN, axis=1)

    return team_players_9cat_stats_dataframe, current_team_9cat_average_stats, \
           new_team_9cat_average_stats_dataframe, new_team_9cat_average_stats


# Format and find 9cat differences
def remove_player_and_float_convert(dataframe):

    for column in dataframe.columns:
        dataframe[column] = dataframe[column].astype(float)

    return dataframe


def drop_add_mean_9cat_difference(current_9cat, new_9cat):

    current_9cat = remove_player_and_float_convert(current_9cat)
    new_9cat = remove_player_and_float_convert(new_9cat)

    difference_9cat = new_9cat.subtract(current_9cat)

    return difference_9cat


# Visualise current, new and difference
def visualise_team_9cat_averages(player_to_drop, player_to_add, current_9cat, new_9cat,
                                 difference_9cat, visualise=True):
    if visualise:
        print(f"You are dropping {player_to_drop} and adding {player_to_add}! \n")

        print("The current team 9cat averages are:\n")
        print(current_9cat)
        print("\n")

        print("The new team 9cat averages are:\n")
        print(new_9cat)
        print("\n")

        print("The difference 9cat averages are:\n")
        print(difference_9cat)
        print("\n")
    else:
        pass


# Function for the entire process
def compare_team_9cats_on_transaction(team, player_to_drop, player_to_add, visualise=True):

    current_players_9cat_averages, current_team_9cat_averages, new_players_9cat_averages, \
    new_team_9cat_averages = waiver_add_and_drop(team, player_to_drop, player_to_add)

    difference_team_9cat_averages = drop_add_mean_9cat_difference(current_team_9cat_averages,
                                                  new_team_9cat_averages)

    visualise_team_9cat_averages(player_to_drop, player_to_add, current_team_9cat_averages,
                             new_team_9cat_averages, difference_team_9cat_averages, visualise)

    return current_players_9cat_averages, current_team_9cat_averages, new_players_9cat_averages, \
           new_team_9cat_averages, difference_team_9cat_averages


# Function power ranking
def power_rankings(dataframe):

    power_ranking_dataframe =  dataframe.copy()
    try:
        power_ranking_dataframe.drop(MEAN_ROW, inplace=True)
    except KeyError:
        pass
    for column in power_ranking_dataframe.columns[1:]:

        sorted_points = power_ranking_dataframe[column].sort_values(ascending=False)
        sorted_points.drop_duplicates(inplace=True)

        sorting_dictionary = dict()

        count = 1
        for point in sorted_points:
            sorting_dictionary[point] = count
            count += 1

        power_ranking_dataframe[column + RANK_SUFFIX] = power_ranking_dataframe[column].map\
            (sorting_dictionary)

    power_ranking_dataframe.drop(dataframe.columns[1:], axis=1, inplace=True)

    return power_ranking_dataframe

In [7]:
sc = yahoo_fantasy_api_authentication()
league = yahoo_fantasy_league(sc)
PLAYER_TO_DROP = "Will Barton"
PLAYER_TO_ADD = "Monte Morris"

current_players_9cat_averages, current_team_9cat_averages, new_players_9cat_averages, \
    new_team_9cat_averages, difference_team_9cat_averages = compare_team_9cats_on_transaction\
    (NUNN_OF_YALL_BETTA, PLAYER_TO_DROP, PLAYER_TO_ADD)

[2021-01-27 23:13:20,502 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-01-27 23:13:20,505 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 555.8798151016235
[2021-01-27 23:13:20,505 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID


You are dropping Will Barton and adding Monte Morris! 

The current team 9cat averages are:

      FG_PCT  FT_PCT  FG3M    PTS   REB   AST   STL  BLK   TOV
mean   0.473   0.826  1.96  15.63  5.04  3.71  1.03  0.4  1.63


The new team 9cat averages are:

      FG_PCT  FT_PCT  FG3M    PTS   REB   AST   STL   BLK   TOV
mean    0.48   0.825  1.96  15.68  4.92  3.72  1.02  0.39  1.58


The difference 9cat averages are:

      FG_PCT  FT_PCT  FG3M   PTS   REB   AST   STL   BLK   TOV
mean   0.007  -0.001   0.0  0.05 -0.12  0.01 -0.01 -0.01 -0.05




## Streamlit
This section creates logic and code needed to run efficiently on Streamlit

In [16]:
def get_team_id_from_team_name(team_name):

    return "".join([x[TEAM_KEY] for x in league_team_list if x[NAME_KEY] == team_name])


def team_player_list(team_id):
    sc = yahoo_fantasy_api_authentication()
    league = yahoo_fantasy_league(sc)
    team = league.to_team(team_id)
    team_roster = team.roster()
    player_in_team_list = [player[NAME_KEY] for player in team_roster]

    return player_in_team_list


def streamlit_return_players_on_team(team_name):

    team_id = get_team_id_from_team_name(team_name)
    player_list = team_player_list(team_id)

    return player_list

In [6]:
team_player_list(NUNN_OF_YALL_BETTA)

[2021-01-29 09:44:17,832 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-01-29 09:44:17,844 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 2453.2029569149017
[2021-01-29 09:44:17,845 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID


['Patty Mills',
 'Will Barton',
 'Malcolm Brogdon',
 'Jeremy Lamb',
 'Khris Middleton',
 'Davis Bertans',
 'Nikola Jokic',
 'Pascal Siakam',
 'Eric Bledsoe',
 'Cameron Johnson',
 'Wayne Ellington',
 'Jimmy Butler',
 'Duncan Robinson',
 'Enes Kanter',
 'Marcus Smart']

In [7]:
league_team_list

[{'team_key': '402.l.55374.t.1', 'name': 'LaLaLand'},
 {'team_key': '402.l.55374.t.2', 'name': 'Autopick'},
 {'team_key': '402.l.55374.t.3', 'name': 'CrabbeHerByThePussy'},
 {'team_key': '402.l.55374.t.4', 'name': "Magic's Johnson"},
 {'team_key': '402.l.55374.t.5', 'name': 'McCurry'},
 {'team_key': '402.l.55374.t.6', 'name': "Nunn of Y'all Betta"},
 {'team_key': '402.l.55374.t.7', 'name': 'Russty Cuntbrooks'},
 {'team_key': '402.l.55374.t.8', 'name': 'Wakanda Forever'},
 {'team_key': '402.l.55374.t.9', 'name': 'Swaggy P'},
 {'team_key': '402.l.55374.t.10', 'name': 'yOBItch Toppin Me'},
 {'team_key': '402.l.55374.t.11', 'name': "Tvon's Tip-Top Team"},
 {'team_key': '402.l.55374.t.12', 'name': 'El Ladrón de Cabras'}]

In [14]:
"".join([x["team_key"] for x in league_team_list if x["name"] == 'LaLaLand'])

'402.l.55374.t.1'

In [17]:
streamlit_return_players_on_team("Autopick")


[2021-01-29 09:52:16,276 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-01-29 09:52:16,277 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 2931.6353845596313
[2021-01-29 09:52:16,277 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID


['LaMelo Ball',
 'Shake Milton',
 'Josh Richardson',
 'Josh Hart',
 'Serge Ibaka',
 'Shai Gilgeous-Alexander',
 'Clint Capela',
 'Paul Millsap',
 'Seth Curry',
 'Bobby Portis',
 'Damian Lillard',
 'Tyler Herro',
 'Ja Morant',
 'Jaylen Brown',
 'Lauri Markkanen']